In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

/Users/sampathweb/miniconda3/envs/py35/lib/python3.5/site-packages/matplotlib/__init__.py:913: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [3]:
import keras
# from keras_tqdm import TQDMNotebookCallback

# from utils import PlotLossNotebookCallback

In [57]:
## keras_preprocessing.py

import os.path
import numpy as np

import keras.backend as K
from keras.preprocessing.image import Iterator, load_img, img_to_array, ImageDataGenerator

from keras.applications import vgg16
import numpy as np

def preprocess_input_vgg16(x):
    """Wrapper around keras.applications.vgg16.preprocess_input()
    to make it compatible for use with keras.preprocessing.image.ImageDataGenerator's
    `preprocessing_function` argument.
    
    Parameters
    ----------
    x : a numpy 3darray (a single image to be preprocessed)
    
    Note we cannot pass keras.applications.vgg16.preprocess_input()
    directly to to keras.preprocessing.image.ImageDataGenerator's
    `preprocessing_function` argument because the former expects a
    4D tensor whereas the latter expects a 3D tensor. Hence the
    existence of this wrapper.
    
    Returns a numpy 3darray (the preprocessed image).
    
    """
    
    X = np.expand_dims(x, axis=0)
    X = vgg16.preprocess_input(X)
    return X[0]

class ImageFilesIterator(Iterator):
    """Iterator capable of reading images from a directory on disk.
    # Arguments
        directory: Path to the directory to read images from.
            Each subdirectory in this directory will be
            considered to contain images from one class,
            or alternatively you could specify class subdirectories
            via the `classes` argument.
        image_data_generator: Instance of `ImageDataGenerator`
            to use for random transformations and normalization.
        target_size: tuple of integers, dimensions to resize input images to.
        color_mode: One of `"rgb"`, `"grayscale"`. Color mode to read images.
        classes: Optional list of strings, names of sudirectories
            containing images from each class (e.g. `["dogs", "cats"]`).
            It will be computed automatically if not set.
        class_mode: Mode for yielding the targets:
            `"binary"`: binary targets (if there are only two classes),
            `"categorical"`: categorical targets,
            `"sparse"`: integer targets,
            `None`: no targets get yielded (only input images are yielded).
        batch_size: Integer, size of a batch.
        shuffle: Boolean, whether to shuffle the data between epochs.
        seed: Random seed for data shuffling.
        data_format: String, one of `channels_first`, `channels_last`.
        save_to_dir: Optional directory where to save the pictures
            being yielded, in a viewable format. This is useful
            for visualizing the random transformations being
            applied, for debugging purposes.
        save_prefix: String prefix to use for saving sample
            images (if `save_to_dir` is set).
        save_format: Format to use for saving sample images
            (if `save_to_dir` is set).
    """

    def __init__(self, directory, filenames, classes,
                 image_data_generator,
                 num_class=None, class_mode='categorical',
                 target_size=(256, 256), color_mode='rgb',
                 batch_size=32, shuffle=True, seed=None,
                 data_format=None,
                 save_to_dir=None, save_prefix='', save_format='jpeg',
                 follow_links=False):
        
        if data_format is None:
            data_format = K.image_data_format()
            
        self.directory = directory
        self.filenames = filenames
        self.classes = classes
        self.num_class = num_class
        self.class_mode = class_mode
        self.samples = len(self.filenames)
        self.image_data_generator = image_data_generator
        self.target_size = tuple(target_size)

        if color_mode not in {'rgb', 'grayscale'}:
            raise ValueError('Invalid color mode:', color_mode,
                             '; expected "rgb" or "grayscale".')
        self.color_mode = color_mode
        self.data_format = data_format
        if self.color_mode == 'rgb':
            if self.data_format == 'channels_last':
                self.image_shape = self.target_size + (3,)
            else:
                self.image_shape = (3,) + self.target_size
        else:
            if self.data_format == 'channels_last':
                self.image_shape = self.target_size + (1,)
            else:
                self.image_shape = (1,) + self.target_size

        self.save_to_dir = save_to_dir
        self.save_prefix = save_prefix
        self.save_format = save_format

        white_list_formats = {'png', 'jpg', 'jpeg', 'bmp'}

        super().__init__(self.samples, batch_size, shuffle, seed)

    def next(self):
        """For python 2.x.
        # Returns
            The next batch.
        """
        with self.lock:
            index_array, current_index, current_batch_size = next(self.index_generator)
        # The transformation of images is not under thread lock
        # so it can be done in parallel
        batch_x = np.zeros((current_batch_size,) + self.image_shape, dtype=K.floatx())
        grayscale = self.color_mode == 'grayscale'
        # build batch of image data
        err_indexes = []
        for i, idx in enumerate(index_array):
            fname = self.filenames[idx]
            try:
                img = load_img(os.path.join(self.directory, fname),
                               grayscale=grayscale,
                               target_size=self.target_size)
                x = img_to_array(img, data_format=self.data_format)
            except Exception:
                err_indexes.append(idx)
                continue
            x = self.image_data_generator.random_transform(x)
            x = self.image_data_generator.standardize(x)
            batch_x[i] = x
            
        
        # optionally save augmented images to disk for debugging purposes
        if self.save_to_dir:
            for i in range(current_batch_size):
                img = array_to_img(batch_x[i], self.data_format, scale=True)
                fname = '{prefix}_{index}_{hash}.{format}'.format(prefix=self.save_prefix,
                                                                  index=current_index + i,
                                                                  hash=np.random.randint(1e4),
                                                                  format=self.save_format)
                img.save(os.path.join(self.save_to_dir, fname))

        # build batch of labels
        if self.class_mode == 'sparse':
            batch_y = self.classes[index_array]
        elif self.class_mode == 'binary':
            batch_y = self.classes[index_array].astype(K.floatx())
        elif self.class_mode == 'categorical':
            batch_y = np.zeros((len(batch_x), self.num_class), dtype=int)
            for i, label in enumerate(self.classes[index_array]):
                batch_y[i, label] = 1.
        else:
            return batch_x
        return batch_x, batch_y
    

class ImageFilesGenerator(ImageDataGenerator):
    
    def flow_from_files(self, directory, 
                            filenames, 
                            classes, num_class=None,
                            target_size=(256, 256), color_mode='rgb',
                            class_mode='categorical',
                            batch_size=32, shuffle=True, seed=None,
                            save_to_dir=None,
                            save_prefix='',
                            save_format='jpeg',
                            follow_links=False,
                            files_iterator_cls=ImageFilesIterator):
        return files_iterator_cls(
            directory, filenames, classes, self, 
            num_class=num_class,
            target_size=target_size, color_mode=color_mode,
            class_mode=class_mode,
            data_format=self.data_format,
            batch_size=batch_size, shuffle=shuffle, seed=seed,
            save_to_dir=save_to_dir,
            save_prefix=save_prefix,
            save_format=save_format,
            follow_links=follow_links)


In [ ]:
from loaders import ImageFilesGenerator, train_test_split, load_index_targets
from keras_preprocessing import preprocess_input_vgg16

In [43]:
# data_loaders.py

import numpy as np
import pandas as pd


def load_labels(filename, index_colname, label_colname):
    
    labels_df = pd.read_csv(filename).set_index(index_colname)
    labels_lookup = labels_df.to_dict(orient="dict")[label_colname]
    return labels_lookup


def train_test_split(X, y, test_size=0.2):
    X = np.array(X)
    y = np.array(y)
    indexes = np.random.permutation(range(len(X)))
    X = X[indexes]
    y = y[indexes]
    
    test_idx = int(len(X) * test_size)
    X_test, y_test = X[:test_idx], y[:test_idx]
    X_train, y_train = X[test_idx:], y[test_idx:]
    
    return (X_train, y_train), (X_test, y_test)

In [59]:
labels_lookup = load_labels("data/category_labels.csv", "image", "category")
images, targets = zip(*labels_lookup.items())

In [60]:
# Key Value of categories
list(labels_lookup.items())[:5]

[('000239.jpg', 4),
 ('000874.jpg', 3),
 ('001376.jpg', 0),
 ('000418.jpg', 4),
 ('000689.jpg', 3)]

In [61]:
images[:5], targets[:5]

(('000239.jpg', '000874.jpg', '001376.jpg', '000418.jpg', '000689.jpg'),
 (4, 3, 0, 4, 3))

In [70]:
# Split the data into Train and Test
(train_images, train_targets), (valid_images, valid_targets) = train_test_split(images, targets)

In [69]:
len(valid_images)

220

In [84]:
batch_size = 32
nb_classes = len(np.unique(train_labels))

train_datagen = ImageFilesGenerator(preprocessing_function=preprocess_input_vgg16,
                                     rotation_range=20,
                                     width_shift_range=0.2,
                                     height_shift_range=0.1,
                                     shear_range=0.1,
                                     zoom_range=0.1,
                                     horizontal_flip=True,
                                     fill_mode="nearest")

train_generator = train_datagen.flow_from_files(
                        "data/ClothingAttributeDataset/images/",
                        train_images, train_targets, 
                        num_class=nb_classes,
                        shuffle=True,
                        target_size=(400, 266),
                        batch_size=batch_size,
                        class_mode="categorical")

In [72]:
x_batches, y_batches = [], []
for idx in range(2):
    x_batch, y_batch = next(train_generator)
    x_batches.append(x_batch)
    y_batches.append(y_batch)
    
x_train = np.concatenate(x_batches)
y_train = np.concatenate(y_batches)

In [73]:
x_train.shape

(64, 400, 266, 3)

In [74]:
y_train.shape

(64, 7)

In [76]:
valid_datagen = ImageFilesGenerator(preprocessing_function=preprocess_input_vgg16)

valid_generator = valid_datagen.flow_from_files(
                        "data/ClothingAttributeDataset/images/",
                        valid_images, valid_targets,                         
                        num_class=nb_classes,
                        shuffle=True,
                        target_size=(400, 266),
                        batch_size=batch_size,
                        class_mode="categorical")

In [77]:
## Build Model

In [78]:
from keras.applications import vgg16
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Flatten
from keras.optimizers import RMSprop

def build_model(input_shape, out_dims):
    
    # Load Pre-Trained Model
    vgg16_model = vgg16.VGG16(weights='imagenet', 
                                   include_top=False,
                                   input_shape=input_shape)
    
    # Freeze all pre-trained Layers
    for layer in vgg16_model.layers:
        layer.trainable = False

    # Take the last Conv. Net output
    vgg16_output = vgg16_model.get_layer("block5_pool").output
        
#     input_features = Input(shape=input_shape)
    x = Flatten()(vgg16_model.output)
    x = Dense(4096, activation="relu")(x)
    x = Dense(4096, activation="relu")(x)    
    outputs = Dense(out_dims, activation="sigmoid")(x)
    
    return Model(inputs=vgg16_model.inputs, outputs=outputs)

In [79]:
model = build_model(input_shape=(400, 266, 3), out_dims=nb_classes)

In [80]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 400, 266, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 400, 266, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 400, 266, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 200, 133, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 200, 133, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 200, 133, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 100, 66, 128)      0         
__________

In [81]:
from keras.optimizers import Adam, SGD

# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [82]:
train_samples = len(train_images)
valid_samples = len(valid_images)

In [83]:
model.fit_generator(train_generator, steps_per_epoch=train_samples // batch_size, epochs=10,
                   validation_data=valid_generator, validation_steps=valid_samples // batch_size
                   )
#                    callbacks=[TQDMNotebookCallback(show_inner=True), PlotLossNotebookCallback()])

Epoch 1/10


KeyboardInterrupt: 

In [ ]:
# x = Conv2D(256, (3,3), padding="same", activation="relu", name="ft_conv1")(block5_conv3)
# x = Conv2D(128, (3,3), padding="same", activation="relu", name="ft_conv2")(block5_conv3)
# x = Conv2D(64, (3,3), padding="same", activation="relu", name="ft_conv3")(block5_conv3)
# x = Conv2D(18, (3,3), padding="same", activation="relu", name="ft_conv4")(block5_conv3)


# prediction = Dense(units=num_classes, activation='softmax', name='output')(fc2)
# model = Model(inputs=vgg16_model.inputs, outputs=prediction)

# for layer in model.layers:
#     if layer.name in ['fc1', 'fc2', 'output']:
#         continue
#     layer.trainable = False

# model.summary()

In [ ]:
# for x_batch, y_batch in train_generator:
#     pretrain_features = vgg16_model.predict(x_batch)
#     ft_model.fit(pretrain_features, y_batch, batch_size=batch_size, )